In [ ]:
import pickle
import pandas as pd
import os
import setup

import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from tqdm.notebook import tqdm, trange
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
import wandb
from torch.utils.data.sampler import SubsetRandomSampler
import pprint
import matplotlib.pyplot as plt
import torch

from sklearn.preprocessing import MinMaxScaler
import math
from torch.profiler import profile, record_function, ProfilerActivity

from operator import itemgetter
import operator
from random import randint
# from rnn_classes import Dog, DogInput, Race, Races, GRUNet, smallGRUNet, smalll_lin_GRUNet, smalll_prelin_GRUNet
import rnn_tools.rnn_classes as rnn_classes
from rnn_tools.raceDB import build_dataset
import importlib
import datetime
from rnn_tools.model_saver import model_saver, model_saver_wandb
import rnn_tools.training_testing_gru as training_testing_gru
from torch.nn.utils.rnn import pack_padded_sequence, pack_sequence, pad_packed_sequence,pad_sequence, unpack_sequence, unpad_sequence
import rnn_tools.training_testing_gru_double as training_testing_gru_double
import rnn_tools.training_testing_lstm as training_testing_lstm
from goto_conversion import goto_conversion

import rnn_tools.training_testing_gru_extra_data as training_testing_gru_extra_data
import rnn_tools.training_testing_gru_ensemble as training_testing_gru_ensemble

In [ ]:
import rnn_tools.rl_grv as rl_grv

In [ ]:
model_name = 'kind-sponge-234'

data = pd.read_feather(F'./model_all_price/RLkind-sponge-234 - val_all_price_df.fth')
test_data =  pd.read_feather(F'./model_all_price/RLkind-sponge-234 - all_price_df.fth')
all_data = pd.concat([data, test_data]) 

In [ ]:
all_data.dog_name.value_counts()

In [ ]:
all_data['pred_price'] = 1/all_data['output_price']

In [ ]:
all_data.columns

In [ ]:
all_data = all_data.sort_values(['raceID','dog_box'])

In [ ]:
all_data.head()

In [ ]:

all_data.win_price.value_counts(dropna=False)

In [ ]:
all_data.dog_box.value_counts()

In [ ]:
all_data.prices = all_data.prices.fillna(0)
all_data['price_probs'] = 1/all_data.prices
print(all_data.price_probs.value_counts(dropna=False))
all_data['price_probs'] = all_data.price_probs.fillna(0).replace([np.inf, -np.inf], 0)
print(all_data.price_probs.value_counts(dropna=False))

In [ ]:
all_data.price_probs

In [ ]:
all_data.columns

In [ ]:
all_data_g = all_data.groupby('raceID')

prices = []
price_preds = []
model_preds = []
dogs = []
results = []
print(all_data.columns)
for name, group in all_data_g:
    # print(group.columns)
    if group['prices'].sum() == 0:
        continue
    prices.append(group['price_probs'].values)
    price_preds.append(group['pred_price'].values)
    model_preds.append(group['pred_prob'].values)
    dogs.append(group['dog_name'].values)
    results.append(group['one_hot_win'].values)

In [ ]:
prices

In [ ]:
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.callbacks import BaseCallback
from wandb.integration.sb3 import WandbCallback 
import rnn_tools.rl_grv as rl_grv
importlib.reload(rl_grv)
import importlib
from stable_baselines3 import A2C
from sklearn.model_selection import train_test_split
from stable_baselines3.common.evaluation import evaluate_policy
#     "policy_type": "MlpPolicy",
# }
# run = wandb.init(
#     project="sb3",
#     config=config,
#     sync_tensorboard=True,  # auto-upload sb3's tensorboard metrics
#     monitor_gym=True,  # auto-upload the videos of agents playing the game
#     # save_code=True,  # optional
# )



dogs_train, dogs_test, prices_train, prices_test, price_preds_train, price_preds_test, model_preds_train, model_preds_test, results_train, results_test = train_test_split(dogs, prices, price_preds, model_preds, results, test_size=0.2, random_state=42)




trainraceData = rl_grv.RaceData(dogs_train,prices_train, price_preds_train, model_preds_train, results_train)
eval_raceData = rl_grv.RaceData(dogs_test,prices_test, price_preds_test, model_preds_test, results_test)
env = rl_grv.GreyhoundRacingEnv(trainraceData)
env = DummyVecEnv([lambda: env])

# Create a separate environment for evaluation
eval_env = rl_grv.GreyhoundRacingEnv(eval_raceData)
eval_env = DummyVecEnv([lambda: eval_env])
from stable_baselines3.common.monitor import Monitor

# eval_env = Monitor(eval_env) 


# Initialize the agent
model = PPO("MlpPolicy", env, verbose=0, learning_rate=0.01)

# model = A2C("MlpPolicy", env, verbose=0)
print(f"{len(eval_raceData.dogs)=} {len(trainraceData.dogs)=}")
# Train model 
# Initialize W&B
wandb.init(project="sb3",save_code=False)
# wandb_callback = WandbCallback()
class CustomWandbCallback(WandbCallback):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.rewards = []
        self.log_interval = 1000
        self.actions = []

    def _on_step(self) -> bool:
        super()._on_step()
        # Append the reward to the list of rewards
        self.rewards.append(self.training_env.get_attr('reward')[0])
        # If the number of steps is a multiple of the log interval, log the average reward
        # self.actions.append(self.training_env.get_attr('action')[0])
        if len(self.rewards) % self.log_interval == 0:
            avg_reward = sum(self.rewards) / len(self.rewards)
            wandb.log({'avg_reward': avg_reward})
            # Clear the list of rewards
            # wandb.log({'action': self.actions})
            
            self.rewards = []
            self.actions = []
        return True

# Use the custom callback
wandb_callback = CustomWandbCallback()


# Number of training iterations
n_iterations = 100

# Number of timesteps to train for each iteration
timesteps_per_iteration = 10_000

# Number of episodes to test for each iteration
test_episodes_per_iteration = 1

for iteration in trange(n_iterations):
    env.reset()
    # Train the model
    model.learn(total_timesteps=timesteps_per_iteration, callback=wandb_callback)

    # Test the model
    episode_rewards = []
    episode_lengths = []
    obs = eval_env.reset()
    done = False
    episode_reward = 0
    episode_length = 0
    while not done:
        action, _ = model.predict(obs, deterministic=True)
        obs, reward, done, info = eval_env.step(action)
        episode_reward += reward
        episode_length += 1
    std_reward = 0
    tqdm.write(f"Mean reward: {episode_reward/episode_length} +/- {std_reward}")
    # print(f"Mean reward: {mean_reward} +/- {std_reward}")

    # Log the rewards to wandb
    wandb.log({'mean_reward': episode_reward/episode_length, 'std_reward': std_reward})


In [ ]:
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.callbacks import BaseCallback
from wandb.integration.sb3 import WandbCallback 
import rnn_tools.rl_grv_v_simple as rl_grv
importlib.reload(rl_grv)
import importlib
from stable_baselines3 import A2C
from sklearn.model_selection import train_test_split
from stable_baselines3.common.evaluation import evaluate_policy
#     "policy_type": "MlpPolicy",
# }
# run = wandb.init(
#     project="sb3",
#     config=config,
#     sync_tensorboard=True,  # auto-upload sb3's tensorboard metrics
#     monitor_gym=True,  # auto-upload the videos of agents playing the game
#     # save_code=True,  # optional
# )



dogs_train, dogs_test, prices_train, prices_test, price_preds_train, price_preds_test, model_preds_train, model_preds_test, results_train, results_test = train_test_split(dogs, prices, price_preds, model_preds, results, test_size=0.2, random_state=42)




trainraceData = rl_grv.RaceData(dogs_train,prices_train, price_preds_train, model_preds_train, results_train)
eval_raceData = rl_grv.RaceData(dogs_test,prices_test, price_preds_test, model_preds_test, results_test)
env = rl_grv.GreyhoundRacingEnv(trainraceData)
env = DummyVecEnv([lambda: env])

# Create a separate environment for evaluation
eval_env = rl_grv.GreyhoundRacingEnv(eval_raceData)
eval_env = DummyVecEnv([lambda: eval_env])
from stable_baselines3.common.monitor import Monitor

# eval_env = Monitor(eval_env) 


# Initialize the agent
model = PPO("MlpPolicy", env, verbose=0, learning_rate=0.01)

# model = A2C("MlpPolicy", env, verbose=0)
print(f"{len(eval_raceData.dogs)=} {len(trainraceData.dogs)=}")
# Train model 
# Initialize W&B
wandb.init(project="sb3",save_code=False)
# wandb_callback = WandbCallback()
class CustomWandbCallback(WandbCallback):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.rewards = []
        self.log_interval = 1000
        self.actions = []

    def _on_step(self) -> bool:
        super()._on_step()
        # Append the reward to the list of rewards
        self.rewards.append(self.training_env.get_attr('reward')[0])
        # If the number of steps is a multiple of the log interval, log the average reward
        self.actions.append(self.training_env.get_attr('action')[0])
        if len(self.rewards) % self.log_interval == 0:
            avg_reward = sum(self.rewards) / len(self.rewards)
            wandb.log({'avg_reward': avg_reward})
            # Clear the list of rewards
            wandb.log({'action': self.actions})
            self.rewards = []
            self.actions = []
        return True

# Use the custom callback
wandb_callback = CustomWandbCallback()


# Number of training iterations
n_iterations = 100

# Number of timesteps to train for each iteration
timesteps_per_iteration = 10_000

# Number of episodes to test for each iteration
test_episodes_per_iteration = 1

for iteration in trange(n_iterations):
    env.reset()
    # Train the model
    model.learn(total_timesteps=timesteps_per_iteration)

    # Test the model
    episode_rewards = []
    episode_lengths = []
    obs = eval_env.reset()
    done = False
    episode_reward = 0
    episode_length = 0
    while not done:
        action, _ = model.predict(obs, deterministic=True)
        # print(eval_env.step(action))
        # print(obs)
        # print(action)
        obs, reward, done, info = eval_env.step(action)
        episode_reward += reward
        episode_length += 1
    std_reward = 0
    tqdm.write(f"Mean reward: {episode_reward/episode_length} +/- {std_reward}")
    # print(f"Mean reward: {mean_reward} +/- {std_reward}")

    # Log the rewards to wandb
    wandb.log({'mean_reward': episode_reward/episode_length, 'std_reward': std_reward})

# eval_env.get_episode_rewards() 
# eval_env.get_episode_lengths()

In [ ]:
env.reset()

In [ ]:
for i,r in enumerate(trainraceData.results):
    if i > 5:
        break
    print(i,r,np.argmax(r))

In [ ]:
from stable_baselines3 import A2C
from stable_baselines3.common.env_util import make_vec_env
import importlib
importlib.reload(rl_grv)
from rnn_tools.rl_grv_v_simple import MaxValueEnv
# Create the vectorized environment
vec_env = make_vec_env(MaxValueEnv, n_envs=1)

# Initialize the model
model = A2C("MlpPolicy", vec_env, verbose=0)

# Number of training iterations
n_iterations = 10

# Number of timesteps to train for each iteration
timesteps_per_iteration = 10_000

# import matplotlib.pyplot as plt 

# reset the environment and see the initial observation
obs = env.reset()
print("The initial observation is {}".format(obs))
random_action = env.action_space.sample()
random_action
for iteration in range(n_iterations):
    # Train the model
    model.learn(total_timesteps=timesteps_per_iteration)

    # Save the model
    model.save(f"a2c_max_value_{iteration}")

# print("Training complete.")